In [2]:
#abfss://Test@onelake.dfs.fabric.microsoft.com/ZinggData.Lakehouse/Files/data.csv
spark.sparkContext.setCheckpointDir("abfss://Zingg@onelake.dfs.fabric.microsoft.com/data.Lakehouse/Files")

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 6, Finished, Available, Finished)

In [3]:
pip install zingg

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 7, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 MB 43.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 62.2 MB/s eta 0:00:00
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyspark 3.5.1.5.4.20240407 requires py4j==0.10.9.7, but you have py4j 0.10.9 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
spark.sparkContext.getCheckpointDir()

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 8, Finished, Available, Finished)

'abfss://Zingg@onelake.dfs.fabric.microsoft.com/data.Lakehouse/Files/b2adeefa-d873-4af7-9780-3af8598f5959'

In [5]:
pip show zingg

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 9, Finished, Available, Finished)

Name: zingg
Version: 0.4.0
Summary: Zingg Entity Resolution, Data Mastering and Deduplication
Home-page: https://github.com/zinggAI/zingg
Author: Zingg.AI
Author-email: sonalgoyal4@gmail.com
License: https://github.com/zinggAI/zingg/blob/main/LICENSE
Location: /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages
Requires: py4j
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tabulate

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 10, Finished, Available, Finished)

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip show tabulate

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 11, Finished, Available, Finished)

Name: tabulate
Version: 0.9.0
Summary: Pretty-print tabular data
Home-page: 
Author: 
Author-email: Sergey Astanin <s.astanin@gmail.com>
License: MIT
Location: /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [8]:
##you can change these to the locations of your choice
##these are the only two settings that need to change
zinggDir = "abfss://Zingg@onelake.dfs.fabric.microsoft.com/data.Lakehouse/Files/models"
modelId = "testModelFebrl"

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 12, Finished, Available, Finished)

In [9]:
## Define constants
MARKED_DIR = zinggDir + "/" + modelId + "/trainingData/marked/"
UNMARKED_DIR = zinggDir + "/" + modelId + "/trainingData/unmarked/"

# Fill these with your specific details
storage_account = "a1a73dc0-3894-4737-b38c-aa7fea437330"  # Replace with your storage account ID
fabric_url = "dfs.fabric.microsoft.com"

# Updated paths for Microsoft Fabric
MARKED_DIR_DBFS = f"abfss://{storage_account}@{fabric_url}{MARKED_DIR}"
UNMARKED_DIR_DBFS = f"abfss://{storage_account}@{fabric_url}{UNMARKED_DIR}"

## Import necessary libraries
import pandas as pd
import numpy as np
import os
import time
import uuid
from tabulate import tabulate
from ipywidgets import widgets, interact, GridspecLayout
import base64
import pyspark.sql.functions as fn

# Import Azure libraries for Fabric
from azure.identity import DefaultAzureCredential
from azure.storage.filedatalake import DataLakeServiceClient

# Zingg libraries
from zingg.client import *
from zingg.pipes import *

# Setup Fabric authentication
def get_service_client():
    credential = DefaultAzureCredential()
    service_client = DataLakeServiceClient(
        account_url=f"https://{storage_account}.dfs.fabric.microsoft.com",
        credential=credential,
    )
    return service_client

service_client = get_service_client()

# Function to clean model directories in Fabric
def cleanModel():
    try:
        # Access the file system
        file_system_client = service_client.get_file_system_client(file_system=storage_account)
        
        # Remove marked directory
        if file_system_client.get_directory_client(MARKED_DIR).exists():
            file_system_client.get_directory_client(MARKED_DIR).delete_directory()
        
        # Remove unmarked directory
        if file_system_client.get_directory_client(UNMARKED_DIR).exists():
            file_system_client.get_directory_client(UNMARKED_DIR).delete_directory()
        
        print("Model cleaned successfully.")
    except Exception as e:
        print(f"Error cleaning model: {str(e)}")
    return

# Function to assign label to a candidate pair
def assign_label(candidate_pairs_pd, z_cluster, label):
    '''
    The purpose of this function is to assign a label to a candidate pair
    identified by its z_cluster value. Valid labels include:
       0 - not matched
       1 - matched
       2 - uncertain
    '''
    # Assign label
    candidate_pairs_pd.loc[candidate_pairs_pd['z_cluster'] == z_cluster, 'z_isMatch'] = label
    return

# Function to count labeled pairs
def count_labeled_pairs(marked_pd):
    '''
    The purpose of this function is to count the labeled pairs in the marked folder.
    '''
    n_total = len(np.unique(marked_pd['z_cluster']))
    n_positive = len(np.unique(marked_pd[marked_pd['z_isMatch'] == 1]['z_cluster']))
    n_negative = len(np.unique(marked_pd[marked_pd['z_isMatch'] == 0]['z_cluster']))

    return n_positive, n_negative, n_total

# Setup interactive widget
available_labels = {
    'No Match': 0,
    'Match': 1,
    'Uncertain': 2
}


StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 13, Finished, Available, Finished)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.


In [10]:
#build the arguments for zingg
args = Arguments()
# Set the modelid and the zingg dir. You can use this as is
args.setModelId(modelId)
args.setZinggDir(zinggDir)
print(args)

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 14, Finished, Available, Finished)

In [11]:
# Import pandas
import pandas as pd

# Define the schema (optional for validation)
schema = ["id", "fname", "lname", "stNo", "add1", "add2", "city", "state", "dob", "ssn"]

# Load the CSV file
data = pd.read_csv("abfss://Zingg@onelake.dfs.fabric.microsoft.com/data.Lakehouse/Files/data.csv")

# Ensure column names match the schema
data.columns = schema  # Adjust only if the file's column names differ

# Display the data
data.head()


StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 15, Finished, Available, Finished)

,id,fname,lname,stNo,add1,add2,city,state,dob,ssn
0,rec-1021-dup-0,thomas,george,1,mcmanus place,stoney creek,3130,sa,19630225,5460534
1,rec-1021-org,thomas,george,1,mcmanus place,north turramurra,3130,sa,19630225,5460534
2,rec-1022-dup-0,jackson,eglinton,840,fowles street,mountview,2803,sa,19830807,2932837
3,rec-1022-dup-1,jackson,eglinton,840,fowles street,moun tjiew,2830,sa,19830807,2932837
4,rec-1022-dup-2,jackson,eglinton,840,fowles street,mou nview,2830,sa,19830807,2932837


In [12]:
schema = "rec_id string, fname string, lname string, stNo string, add1 string, add2 string, city string, state string, dob string, ssn string"
inputPipe = CsvPipe("testFebrl", "abfss://Zingg@onelake.dfs.fabric.microsoft.com/data.Lakehouse/Files/data.csv", schema)

args.setData(inputPipe)

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 16, Finished, Available, Finished)

set schema 


In [13]:
#setting outputpipe in 'args'
outputPipe = CsvPipe("resultOutput", "abfss://Zingg@onelake.dfs.fabric.microsoft.com/data.Lakehouse/Files")
args.setOutput(outputPipe)

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 17, Finished, Available, Finished)

In [14]:
# Set field definitions
rec_id = FieldDefinition("rec_id", "string", MatchType.EXACT)        # ID should use exact match
fname = FieldDefinition("fname", "string", MatchType.FUZZY)  # First Name
lname = FieldDefinition("lname", "string", MatchType.FUZZY)  # Last Name
stNo = FieldDefinition("stNo", "string", MatchType.FUZZY)    # Street Number
add1 = FieldDefinition("add1", "string", MatchType.FUZZY)    # Address Line 1
add2 = FieldDefinition("add2", "string", MatchType.FUZZY)    # Address Line 2
city = FieldDefinition("city", "string", MatchType.FUZZY)    # City
state = FieldDefinition("state", "string", MatchType.FUZZY)  # State
dob = FieldDefinition("dob", "string", MatchType.EXACT)      # Date of Birth (prefer exact match)
ssn = FieldDefinition("ssn", "string", MatchType.EXACT)      # SSN (should use exact match)

# Create the field definitions list
fieldDefs = [rec_id, fname, lname, stNo, add1, add2, city, state, dob, ssn]

# Set field definitions in args
args.setFieldDefinition(fieldDefs)

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 18, Finished, Available, Finished)

In [15]:
# The numPartitions define how data is split across the cluster. 
# Please change the fllowing as per your data and cluster size by referring to the docs.

args.setNumPartitions(4)
args.setLabelDataSampleSize(0.5)

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 19, Finished, Available, Finished)

In [16]:
options = ClientOptions([ClientOptions.PHASE,"findTrainingData"])

#Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
print(args)
print(options)
print(zingg)
zingg.initAndExecute()

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 20, Finished, Available, Finished)

['--phase', 'findTrainingData']
arguments for client options are  ['--phase', 'findTrainingData', '--license', 'zinggLic.txt', '--email', 'zingg@zingg.ai', '--conf', 'dummyConf.json']


In [17]:
options = ClientOptions([ClientOptions.PHASE,"label"])

#Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
zingg.init()

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 21, Finished, Available, Finished)

['--phase', 'label']
arguments for client options are  ['--phase', 'label', '--license', 'zinggLic.txt', '--email', 'zingg@zingg.ai', '--conf', 'dummyConf.json']


In [18]:
# get candidate pairs
candidate_pairs_pd = getPandasDfFromDs(zingg.getUnmarkedRecords())
 
# if no candidate pairs, run job and wait
if candidate_pairs_pd.shape[0] == 0:
  print('No unlabeled candidate pairs found.  Run findTraining job ...')

else:
    # get list of pairs (as identified by z_cluster) to label 
    z_clusters = list(np.unique(candidate_pairs_pd['z_cluster'])) 

    # identify last reviewed cluster
    last_z_cluster = '' # none yet

    # print candidate pair stats
    print('{0} candidate pairs found for labeling'.format(len(z_clusters)))

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 22, Finished, Available, Finished)

15 candidate pairs found for labeling


In [19]:
# Label Training Set

# define variable to avoid duplicate saves
ready_for_save = False
print(candidate_pairs_pd)

# user-friendly labels and corresponding zingg numerical value
# (the order in the dictionary affects how displayed below)
LABELS = {
  'Uncertain':2,
  'Match':1,
  'No Match':0  
  }

# GET CANDIDATE PAIRS
# ========================================================
#candidate_pairs_pd = get_candidate_pairs()
n_pairs = int(candidate_pairs_pd.shape[0]/2)
# ========================================================

# DEFINE IPYWIDGET DISPLAY
# ========================================================
display_pd = candidate_pairs_pd.drop(
  labels=[
    'z_zid', 'z_prediction', 'z_score', 'z_isMatch', 'z_zsource'
    ], 
  axis=1)

# define header to be used with each displayed pair
html_prefix = "<p><span style='font-family:Courier New,Courier,monospace'>"
html_suffix = "</p></span>"
header = widgets.HTML(value=f"{html_prefix}<b>" + "<br />".join([str(i)+"&nbsp;&nbsp;" for i in display_pd.columns.to_list()]) + f"</b>{html_suffix}")

# initialize display
vContainers = []
vContainers.append(widgets.HTML(value=f'<h2>Indicate if each of the {n_pairs} record pairs is a match or not</h2></p>'))

# for each set of pairs
for n in range(n_pairs):

  # get candidate records
  candidate_left = display_pd.loc[2*n].to_list()
  print(candidate_left)
  candidate_right = display_pd.loc[(2*n)+1].to_list()
  print(candidate_right)

  # define grid to hold values
  html = ''

  for i in range(display_pd.shape[1]):

    # get column name
    column_name = display_pd.columns[i]

    # if field is image
    if column_name == 'image_path':

      # define row header
      html += '<tr>'
      html += '<td><b>image</b></td>'

      # read left image to encoded string
      l_endcode = ''
      if candidate_left[i] != '':
        with open(candidate_left[i], "rb") as l_file:
          l_encode = base64.b64encode( l_file.read() ).decode()

      # read right image to encoded string
      r_encode = ''
      if candidate_right[i] != '':
        with open(candidate_right[i], "rb") as r_file:
          r_encode = base64.b64encode( r_file.read() ).decode()      

      # present images
      html += f'<td><img src="data:image/png;base64,{l_encode}"></td>'
      html += f'<td><img src="data:image/png;base64,{r_encode}"></td>'
      html += '</tr>'

    elif column_name != 'image_path':  # display text values

      if column_name == 'z_cluster': z_cluster = candidate_left[i]

      html += '<tr>'
      html += f'<td style="width:10%"><b>{column_name}</b></td>'
      html += f'<td style="width:45%">{str(candidate_left[i])}</td>'
      html += f'<td style="width:45%">{str(candidate_right[i])}</td>'
      html += '</tr>'

  # insert data table
  table = widgets.HTML(value=f'<table data-title="{z_cluster}" style="width:100%;border-collapse:collapse" border="1">'+html+'</table>')
  z_cluster = None

  # assign label options to pair
  label = widgets.ToggleButtons(
    options=LABELS.keys(), 
    button_style='info'
    )

  # define blank line between displayed pair and next
  blankLine=widgets.HTML(value='<br>')

  # append pair, label and blank line to widget structure
  vContainers.append(widgets.VBox(children=[table, label, blankLine]))

# present widget
display(widgets.VBox(children=vContainers))
# ========================================================

# mark flag to allow save 
ready_for_save = True


StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 23, Finished, Available, Finished)

    z_zid         z_cluster  z_prediction  z_score  z_isMatch          rec_id  \
0      34   1734014375837:0          -1.0      0.0         -1  rec-1022-dup-1   
1      17   1734014375837:0          -1.0      0.0         -1  rec-1029-dup-1   
2      56   1734014375837:1          -1.0      0.0         -1  rec-1032-dup-0   
3      26   1734014375837:1          -1.0      0.0         -1  rec-1032-dup-0   
4      47  1734014375837:12          -1.0      0.0         -1  rec-1029-dup-1   
5      17  1734014375837:12          -1.0      0.0         -1  rec-1029-dup-1   
6      59  1734014375837:16          -1.0      0.0         -1    rec-1034-org   
7      29  1734014375837:16          -1.0      0.0         -1    rec-1034-org   
8      32   1734014375837:2          -1.0      0.0         -1    rec-1021-org   
9       2   1734014375837:2          -1.0      0.0         -1    rec-1021-org   
10     33   1734014375837:3          -1.0      0.0         -1  rec-1022-dup-0   
11      3   1734014375837:3 

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 24, Finished, Available, Finished)

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 25, Finished, Available, Finished)

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 26, Finished, Available, Finished)

...

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 36, Finished, Available, Finished)

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 37, Finished, Available, Finished)

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 38, Finished, Available, Finished)

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 39, Finished, Available, Finished)

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 40, Finished, Available, Finished)

In [1]:
notebookutils.fs.ls("/")

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 5, Finished, Available, Finished)

[FileInfo(path=abfss://e803987a-98b6-445f-815c-3d15c2c46877@onelake.dfs.fabric.microsoft.com/36ef8bc2-c67a-4512-b060-e25489729c71, name=36ef8bc2-c67a-4512-b060-e25489729c71, size=0)]

In [20]:
if not ready_for_save:
  print('No labels have been assigned. Run the previous cell to create candidate pairs and assign labels to them before re-running this cell.')

else:

  # ASSIGN LABEL VALUE TO CANDIDATE PAIRS IN DATAFRAME
  # ========================================================
  # for each pair in displayed widget
  for pair in vContainers[1:]:

    # get pair and assigned label
    html_content = pair.children[1].get_interact_value() # the displayed pair as html
    user_assigned_label = pair.children[1].get_interact_value() # the assigned label

    # extract candidate pair id from html pair content
    start = pair.children[0].value.find('data-title="')
    if start > 0: 
      start += len('data-title="') 
      end = pair.children[0].value.find('"', start+2)
    pair_id = pair.children[0].value[start:end]



    # assign label to candidate pair entry in dataframe
    candidate_pairs_pd.loc[candidate_pairs_pd['z_cluster']==pair_id, 'z_isMatch'] = LABELS.get(user_assigned_label)
  # ========================================================

  # SAVE LABELED DATA TO ZINGG FOLDER
  # ========================================================
  # make target directory if needed
  notebookutils.fs.mkdirs(MARKED_DIR)
  
  # save label assignments
  # save labels
  zingg.writeLabelledOutputFromPandas(candidate_pairs_pd,args)

  # count labels accumulated
  marked_pd_df = getPandasDfFromDs(zingg.getMarkedRecords())
  n_pos, n_neg, n_tot = count_labeled_pairs(marked_pd_df)
  print(f'You have accumulated {n_pos} pairs labeled as positive matches.')
  print(f'You have accumulated {n_neg} pairs labeled as not matches.')
  print("If you need more pairs to label, re-run the cell for 'findTrainingData'")
  # ========================================================  

  # save completed
  ready_for_save = False

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 41, Finished, Available, Finished)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


You have accumulated 9 pairs labeled as positive matches.
You have accumulated 6 pairs labeled as not matches.
If you need more pairs to label, re-run the cell for 'findTrainingData'


In [21]:
options = ClientOptions([ClientOptions.PHASE,"trainMatch"])

#Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
zingg.initAndExecute()

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 42, Finished, Available, Finished)

['--phase', 'trainMatch']
arguments for client options are  ['--phase', 'trainMatch', '--license', 'zinggLic.txt', '--email', 'zingg@zingg.ai', '--conf', 'dummyConf.json']


In [26]:
outputDF = spark.read.csv("abfss://Zingg@onelake.dfs.fabric.microsoft.com/data.Lakehouse/Files/part-00000-d624fac4-b80c-4f8d-aebc-5d5faf351b8f-c000.csv")

colNames = ["z_minScore", "z_maxScore", "z_cluster", "rec_id", "fname", "lname", "stNo", "add1", "add2", "city", "state", "dob", "ssn"]
outputDF.toDF(*colNames).show(100)

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 47, Finished, Available, Finished)

+------------------+------------------+---------+--------------+--------+-------------+----+------------------+----------------+----+-----+--------+-------+
|        z_minScore|        z_maxScore|z_cluster|        rec_id|   fname|        lname|stNo|              add1|            add2|city|state|     dob|    ssn|
+------------------+------------------+---------+--------------+--------+-------------+----+------------------+----------------+----+-----+--------+-------+
|0.9999999999995524|0.9999999999995524|       26|rec-1032-dup-0|brooklyn|naar-caftenas| 210|      duffy street|    tourist park|2481|  nsw|19840802|3624304|
|0.9999999999995358|0.9999999999995358|       24|rec-1031-dup-0|samantha|     sabieray|  68|   quandong street|     wattle brae|4019|   wa|19590807|2863290|
|0.9999999977273273|0.9999999977273273|        2|  rec-1021-org|  thomas|       george|   1|     mcmanus place|north turramurra|3130|   sa|19630225|5460534|
|0.9999999999997746|0.9999999999997746|       15|  rec-102

In [27]:
options = ClientOptions([ClientOptions.PHASE,"generateDocs"])

#Zingg execution for the given phase
zingg = ZinggWithSpark(args, options)
zingg.initAndExecute()

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 48, Finished, Available, Finished)

['--phase', 'generateDocs']
arguments for client options are  ['--phase', 'generateDocs', '--license', 'zinggLic.txt', '--email', 'zingg@zingg.ai', '--conf', 'dummyConf.json']


In [29]:
DOCS_DIR = zinggDir + "/" + modelId + "/docs/"

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 50, Finished, Available, Finished)

In [30]:
displayHTML(open(DOCS_DIR+"model.html", 'r').read())

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 51, Finished, Available, Finished)

In [31]:
displayHTML(open(DOCS_DIR+"data.html", 'r').read())

StatementMeta(, e8d52d7f-1f5d-4897-a638-4465746c84f8, 52, Finished, Available, Finished)

Field Name,Field Type,Nullable
rec_id,StringType,true
fname,StringType,true
lname,StringType,true
stNo,StringType,true
add1,StringType,true
add2,StringType,true
city,StringType,true
state,StringType,true
dob,StringType,true
ssn,StringType,true
